# Metadata

**L1 Taxonomy** - Security

**L2 Taxonomy** - Authorization

**Subtopic** - Implementing a simple policy engine reading rules from config and enforcing them in code

**Use Case** - Develop a Python module that reads a set of authorization rules from a JSON configuration file and enforces them. The module should provide a function that takes a user role and an action, and returns whether the action is allowed for that role based on the rules in the configuration file.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- You are tasked with developing a Python module that enforces role-based access control (RBAC) by reading authorization rules from a JSON configuration.
- The module must provide a function that determines whether a given user role is permitted to perform a specific action based on the configured rules.

Function signature:
```python
def load_authorization_rules(config: dict) -> None:
    """Load the authorization rules from the given configuration."""

def is_action_allowed(role: str, action: str) -> bool:
    """Return True if the role is allowed to perform the action, else False."""
```

Input Format:
- A dictionary representing the authorization rules configuration:
```python
{
    "admin": ["create", "read", "update", "delete"],
    "editor": ["read", "update"],
    "viewer": ["read"]
}
```
- Each key is a string representing a user role.
- Each value is a list of strings representing the actions allowed for that role.

Input Constraints:
- The configuration dictionary must contain only:
 - String keys representing user roles.
 - Lists of strings representing allowed actions for each role.
- Each action in the list must be a non-empty string.
- Role names and action names are case-sensitive.
- No role may appear more than once in the configuration.
- The configuration must be loaded once before checking any permissions.
- The role and action parameters passed to is_action_allowed must be strings.

Output Format:
- The function is_action_allowed(role: str, action: str) returns a single value:
 - True   # if the action is allowed for the given role
 - False  # if the action is not allowed or the role is undefined
- Output is always of type bool.

Example:
```python
Input:
config = {
    "admin": ["create", "read", "update", "delete"],
    "editor": ["read", "update"],
    "viewer": ["read"]
}

load_authorization_rules(config)

is_action_allowed("editor", "update")

Output:
True
```

# Requirements
Explicit Requirements:
- The module must read authorization rules from a given configuration dictionary.
- The function is_action_allowed(role, action) must return True or False.
- Role names and action names are case-sensitive.
- The function must return False if the role is not defined in the configuration.
- The configuration must be loaded using load_authorization_rules(config) before checking permissions.

Implicit Requirements:
- The configuration should not contain duplicate role entries.
- An empty list of actions for a role means that role cannot perform any actions.
- If the configuration is empty, all permission checks must return False.
- Both role and action inputs to is_action_allowed must be strings.

Solution Expectations:
- Implement a function load_authorization_rules(config) to load the access rules.
- Implement a function is_action_allowed(role, action) that returns True or False.
- Ensure that the functions handle missing roles or actions gracefully.
- Maintain internal state to store the loaded configuration for future checks.
- Use only built-in Python libraries; no external dependencies are allowed.

Edge Cases and Behavior:
- If the role does not exist in the configuration, return False.
- If the action is not listed for a valid role, return False.
- If the configuration is empty, all permission checks must return False.
- If a role has an empty list of actions, return False for all action checks.
- Role and action inputs must be treated as case-sensitive (e.g., "Admin" ≠ "admin").

Solution Constraints:
- The configuration must be loaded exactly once before permission checks.
- All role and action names must be strings and treated case-sensitively.
- The configuration must be stored in memory for efficient lookups.
- The is_action_allowed function must not modify the configuration.
- Perform input validation on all inputs.

In [ ]:
# code
"""This module provides role-based access control (RBAC) validation."""


_access_rules_data: dict[str, list[str]] = {}


def load_authorization_rules(config: dict) -> None:
    """Load the authorization rules from the given configuration.

    Args:
        config (dict): A dictionary mapping role names (strings) to lists of
            allowed actions (strings).

    Raises:
        TypeError: If the configuration or any of its components are of an
            incorrect type.
        ValueError: If any role name or action is an empty string.
    """
    if not isinstance(config, dict):
        raise TypeError("Configuration must be a dictionary.")

    staging_rules: dict[str, list[str]] = {}

    for role_key, actions_value in config.items():
        if not isinstance(role_key, str) or not role_key:
            raise ValueError(
                f"Role name must be a non-empty string. "
                f"Invalid role: '{role_key}'"
            )

        if not isinstance(actions_value, list):
            raise TypeError(
                f"Actions for role '{role_key}' must be a list. "
                f"Received: {type(actions_value).__name__}"
            )

        valid_actions_for_current_role: list[str] = []

        for individual_action in actions_value:
            if not isinstance(individual_action, str) or not individual_action:
                raise ValueError(
                    f"Action for role '{role_key}'"
                    f" must be a non-empty string. "
                    f"Invalid action: '{individual_action}'"
                )
            valid_actions_for_current_role.append(individual_action)

        staging_rules[role_key] = valid_actions_for_current_role

    global _access_rules_data
    _access_rules_data = staging_rules


def is_action_allowed(role: str, action: str) -> bool:
    """Check if a given role is permitted to perform a specific action.

    Args:
        role (str): The name of the user role.
        action (str): The action to be checked.

    Returns:
        bool: True if the role is allowed to
        perform the action, False otherwise.

    Raises:
        TypeError: If the 'role' or 'action' arguments are not strings.
    """
    if not isinstance(role, str):
        raise TypeError(
            f"The 'role' argument must be a string. "
            f"Received type: {type(role).__name__}"
        )

    if not isinstance(action, str):
        raise TypeError(
            f"The 'action' argument must be a string. "
            f"Received type: {type(action).__name__}"
        )

    if not role or not action:
        return False

    allowed_actions_for_role = _access_rules_data.get(role, [])
    return action in allowed_actions_for_role


if __name__ == "__main__":
    config = {
        "admin": ["create", "read", "update", "delete"],
        "editor": ["read", "update"],
        "viewer": ["read"]
    }

    load_authorization_rules(config)

    print(is_action_allowed("editor", "update"))       # True
    print(is_action_allowed("viewer", "update"))       # False
    print(is_action_allowed("nonexistent", "read"))    # False
    print(is_action_allowed("admin", "delete"))        # True
    print(is_action_allowed("admin", "nonexistent"))   # False
    print(is_action_allowed("", "read"))               # False
    print(is_action_allowed("editor", ""))             # False


In [ ]:
# tests

"""Test module for RBAC system"""

import unittest
from main import load_authorization_rules, is_action_allowed


class TestRBACAuthorization(unittest.TestCase):
    """Unit tests for role-based access control functionality."""

    def setUp(self):
        """Load a base configuration for common tests."""
        self.config = {
            "admin": ["create", "read", "update", "delete"],
            "editor": ["read", "update"],
            "viewer": ["read"],
            "guest": []
        }
        load_authorization_rules(self.config)

    def test_admin_has_all_permissions(self):
        """Test that admin can perform all CRUD operations."""
        self.assertTrue(is_action_allowed("admin", "create"))
        self.assertTrue(is_action_allowed("admin", "read"))
        self.assertTrue(is_action_allowed("admin", "update"))
        self.assertTrue(is_action_allowed("admin", "delete"))

    def test_editor_has_read_and_update(self):
        """Test that editor has access to read and update only."""
        self.assertTrue(is_action_allowed("editor", "read"))
        self.assertTrue(is_action_allowed("editor", "update"))

    def test_editor_cannot_create_or_delete(self):
        """Test that editor cannot create or delete."""
        self.assertFalse(is_action_allowed("editor", "create"))
        self.assertFalse(is_action_allowed("editor", "delete"))

    def test_viewer_can_only_read(self):
        """Test that viewer can only perform the read action."""
        self.assertTrue(is_action_allowed("viewer", "read"))
        self.assertFalse(is_action_allowed("viewer", "update"))
        self.assertFalse(is_action_allowed("viewer", "delete"))

    def test_guest_has_no_permissions(self):
        """Test that a role with empty permissions returns False."""
        self.assertFalse(is_action_allowed("guest", "read"))
        self.assertFalse(is_action_allowed("guest", "create"))

    def test_undefined_role_returns_false(self):
        """Test that an undefined role always returns False."""
        self.assertFalse(is_action_allowed("superuser", "read"))

    def test_undefined_action_for_valid_role_returns_false(self):
        """Test that a valid role cannot perform undefined action."""
        self.assertFalse(is_action_allowed("viewer", "publish"))

    def test_case_sensitivity_of_role_names(self):
        """Test that role names are case-sensitive."""
        self.assertFalse(is_action_allowed("Admin", "read"))
        self.assertTrue(is_action_allowed("admin", "read"))

    def test_case_sensitivity_of_action_names(self):
        """Test that action names are case-sensitive."""
        self.assertFalse(is_action_allowed("admin", "Read"))
        self.assertTrue(is_action_allowed("admin", "read"))

    def test_empty_configuration_denies_all_access(self):
        """Test behavior when empty configuration is loaded."""
        load_authorization_rules({})
        self.assertFalse(is_action_allowed("admin", "read"))
        self.assertFalse(is_action_allowed("editor", "update"))

    def test_new_config_overwrites_previous_one(self):
        """Test that loading a new config replaces the old one."""
        new_config = {"user": ["view"]}
        load_authorization_rules(new_config)
        self.assertTrue(is_action_allowed("user", "view"))
        self.assertFalse(is_action_allowed("admin", "read"))

    def test_role_with_only_one_action(self):
        """Test a role that is allowed to perform only one action."""
        config = {"analyst": ["read"]}
        load_authorization_rules(config)
        self.assertTrue(is_action_allowed("analyst", "read"))
        self.assertFalse(is_action_allowed("analyst", "update"))

    def test_action_is_empty_string(self):
        """Test that empty string as action returns False."""
        self.assertFalse(is_action_allowed("admin", ""))

    def test_role_is_empty_string(self):
        """Test that empty string as role returns False."""
        self.assertFalse(is_action_allowed("", "read"))

    def test_invalid_role_and_action_combination(self):
        """Test a combination of undefined role and action."""
        self.assertFalse(is_action_allowed("random", "execute"))


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/e98475ca-823e-411d-9ca8-d735ca2067f9?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F74374%2Fview

#### Model code:

```python
# code generated by the model
# Module-level variable to store the authorization configuration
authorization_rules = {}

def load_authorization_rules(config: dict) -> None:
    """Load the authorization rules from the given configuration."""
    # <Issue>: Missing input validation
    global authorization_rules
    # Assign the provided configuration to the module-level dictionary
    authorization_rules = config

def is_action_allowed(role: str, action: str) -> bool:
    """Return True if the role is allowed to perform the action, else False."""
    # Check if role is present in the authorization_rules
    # <Issue>: Missing input validation
    if role not in authorization_rules:
        # Role is not defined, return False
        return False
    
    # Check if action is allowed for the role
    allowed_actions = authorization_rules[role]
    return action in allowed_actions

# Test Example:
if __name__ == "__main__":
    config = {
        "admin": ["create", "read", "update", "delete"],
        "editor": ["read", "update"],
        "viewer": ["read"]
    }

    load_authorization_rules(config)

    print(is_action_allowed("editor", "update"))  # Expected output: True
    print(is_action_allowed("viewer", "update"))  # Expected output: False
    print(is_action_allowed("nonexistent", "read"))  # Expected output: False
    print(is_action_allowed("admin", "delete"))  # Expected output: True
    print(is_action_allowed("admin", "nonexistent"))  # Expected output: False
    print(is_action_allowed("", "read"))  # Expected output: False
    print(is_action_allowed("editor", ""))  # Expected output: False
```